In [ ]:
"""
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
"""

In [37]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
name   ="H1_2008-2022_DataFrame"
#df=pd.read_csv("EURUSD.csv")
df = pd.read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python',index_col=[0])
    
#print("Number of rows and columns:", df.shape)
#df = df.drop('Adj Close',axis=1)
#df = df.drop('Volume',axis=1)
df

,Date,Open,High,Low,Close
0,2007-12-31 01:00:00,1.47350,1.47380,1.47320,1.47360
1,2007-12-31 02:00:00,1.47350,1.47470,1.47320,1.47360
2,2007-12-31 03:00:00,1.47370,1.47440,1.47350,1.47440
3,2007-12-31 04:00:00,1.47430,1.47470,1.47420,1.47440
4,2007-12-31 05:00:00,1.47450,1.47470,1.47420,1.47420
...,...,...,...,...,...
86744,2021-12-31 19:00:00,1.13790,1.13861,1.13769,1.13860
86745,2021-12-31 20:00:00,1.13860,1.13860,1.13745,1.13842
86746,2021-12-31 21:00:00,1.13842,1.13853,1.13793,1.13822
86747,2021-12-31 22:00:00,1.13822,1.13847,1.13752,1.13755


In [55]:
train_size = ((len(df)*90)//100)

df_train = df.iloc[:train_size, 1:].values
df_test  = df.iloc[train_size:, 1:].values
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [61]:
def ModelSet(model):
  #Adding the first LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 1024*3, return_sequences = True, input_shape = (X_train.shape[1], 1)))
  #model.add(Dropout(0.33))
  # Adding a second LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 1024*3, return_sequences = True))
  #model.add(Dropout(0.33))
  # Adding a third LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 1024*3, return_sequences = True))
  #model.add(Dropout(0.33))
  # Adding a fourth LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 1024*3))
  model.add(Dropout(0.33))
  # Adding the output layer
  model.add(Dense(units = 4))

  # Compiling the RNN
  model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])
  model.summary()
  # Fitting the RNN to the Training set


In [ ]:
#model.save('EURUSD_M1_CalisanModel/')

In [ ]:
p =0
def Run(df_t):
  total = len(df_t)
  percent = (total//2160)
  
  model = Sequential(name="humes")
  i =0
  for p in range(percent):
    if i <= total:
      df_partial =df_t[i:(2160)+i]
      i = percent + i

    df_y = df_partial.shift(-1, axis = 0)
    df_y =df_y.dropna()

    df_x = df_partial.shift(1,axis=0)
    df_x = df_x.dropna()

    df_x = pd.DataFrame(df_x)
    df_y = pd.DataFrame(df_y)

    df_x,df_y

    from sklearn.model_selection import train_test_split
    #train_size = ((len(df_x)*90)//100)

    X_train, X_test, y_train, y_test = train_test_split(df_x.iloc[:, 0:], df_y.iloc[:, 0:],shuffle=False)
    #print(X_train.shape)
    #X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33,shuffle=False)
    #X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape 

    try:
      model.fit(X_train, y_train, epochs = 1, batch_size = 4096)
    except:
      print("Model Sıfırdan oluşturuluyor")
      #ModelSet(model)
      model.fit(X_train, y_train, epochs = 1, batch_size = 4096)
    #predicted_stock_price = model.predict(X_test)
    print(p,i)  
    #print(predicted_stock_price,"\n",y_test,"\n",p,"\n")

Run(df_train)

Model Sıfırdan oluşturuluyor
Model: "humes"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_61 (LSTM)              (None, 4, 3072)           37773312  
                                                                 
 lstm_62 (LSTM)              (None, 4, 3072)           75509760  
                                                                 
 lstm_63 (LSTM)              (None, 4, 3072)           75509760  
                                                                 
 lstm_64 (LSTM)              (None, 3072)              75509760  
                                                                 
 dropout_13 (Dropout)        (None, 3072)              0         
                                                                 
 dense_13 (Dense)            (None, 4)                 12292     
                                                                 
Total params: 264,314,884
Traina